In [1]:
#!/usr/bin/env/ python3

'''
 This code will help you out in generating Single Frequency Filtering
 Cepstral Coefficients (SFFCC), in a classical approach
'''

import numpy as np
import scipy.io.wavfile as wav
import io
import python_speech_features as py_sp_feat

from scipy.signal import hilbert
from scipy.fftpack import fft
from scipy.fftpack.realtransforms import dct
from python_speech_features import *
from scipy.signal import *
import matplotlib.pyplot as plot


import warnings
warnings.filterwarnings("ignore")



C:\Users\kesav\AppData\Local\Temp\ipykernel_17476\2386552825.py:15: DeprecationWarning: Please use `dct` from the `scipy.fftpack` namespace, the `scipy.fftpack.realtransforms` namespace is deprecated.
  from scipy.fftpack.realtransforms import dct


In [5]:
# ---- Strat code ----- #


def my_resample(ip,fs_old,fs_new):

    if fs_old == fs_new:
        ip = ip
    else:
        ip = resample(ip,fs_new)

    ip = ip/np.max(ip)

    return ip

def transfer_function(r, z):
    '''
    this function is used to generate transfer function
    which is nothing but single pole filtering
    '''

    num = 1
    power = (z)**-1
    den = 1+r*power

    ratio = num/den

    return ratio


def my_sff(file, stepsize, f1, r,fs_new):
    '''
    file we should give the audiofile location
    stepsize is shift in frequency
    f1 is start frequency
    r is pole location which is ideally  r == 1 but in my case r is 0.984
    fs_new is the desired sampling for performing the task
    '''

    # -------- Start SFF code ------- #

    fs_old, x = wav.read(file)                      # Reading the audiofile
    x = my_resample(x,fs_old,fs_new)                # resampling the data as per our requirement
    fs = fs_new
    x = sigproc.preemphasis(x,0.97)                 # Preemphasis of a signal

    # Default parameters to be considered

    delta_f = stepsize
    strt_f = f1
    f2 = int(fs/2)
    stop_f = f2
#///////////////////////////////////////////////////
    samplingFrequency=16000
    plot_spectrogram(x,samplingFrequency) 
#////////////////////////////////////////////////////  


    # Below we are calculating fk

    fk = [i for i in range(f2) if i % delta_f == 0]
    fk = np.array(fk)

    # below we are calculating omega_k

    factor = (fk/fs)
    aa = 2*np.pi
    omega_k = (aa*factor)
    omega_k= [np.pi-x  for x in omega_k]  #changed from omega_k to pi-omega_k
    len_omega_k = len(omega_k)

    # Below lines of the will generate SFF evelope

    env = []
    yk = []

    for j in range(len_omega_k):

        z_power = np.exp(1j*omega_k[j])

        filter_val = transfer_function(r,z_power)   # transfer function (single pole filter)

        filtered_sig = filter_val * x               # filter signal

        abs_filtered_sig = np.absolute(filtered_sig)

        hilbert_sig = hilbert(abs_filtered_sig)

        abs_hilbert_sig = np.absolute(hilbert_sig)

        yk.insert(j,abs_hilbert_sig)

    env = np.transpose(yk)                          # evelope has been generated

    wk_mean = []

    for i in range(len(env[1])):
        wk_mean.append(np.mean(env[:,i])/np.mean(np.mean(env[:,i])))  # this loop is meant for calculating mean for each frequency

    wts = []
    for l in range(len(wk_mean)):
        wts.insert(l,wk_mean[l]*env[:,l])       # Heree weights are being multiplied by weighted components envelopes of speech signal

    return wts


    # ----- End of SFF code -----#
       

def my_cepstral_coeff(signal, nceps):

    nl = np.log10(signal)                                   # non-linearity
    ceps = dct(nl, type=2, norm='ortho', axis=-1)[:nceps]   # Copmression
    return ceps
def plot_spectrogram(s2,samplingFrequency):
    plot.subplot(212)
    plot.specgram(s2,Fs=samplingFrequency)
    plot.xlabel('Time')
    plot.ylabel('Frequency')
    plot.show()   
    

# ----- End of code -------#

